# Microsoft Graph Template

## Initialization

### Log4JS
- [NPM](https://www.npmjs.com/package/log4js)
- [GitHub](https://www.npmjs.com/package/log4js)

In [ ]:
process.env.LOG4JS_DEFAULT_LEVEL = 'debug';
import * as log4js from "log4js";
import {
  getPackageRelativeFilename,
  defaultLog4JSConfiguration,
  getPackageRoot,
} from "@iamsupercb/common";
var logger = log4js.getLogger(getPackageRelativeFilename('jupyter'));
log4js.configure(defaultLog4JSConfiguration);
logger.debug('==  begins ==');
logger.debug('debug');
logger.info('info');
logger.warn('warn');
logger.error('error');
logger.fatal('fatal');
logger.debug('==  ends ==');

### Environment variables

In [ ]:
logger.debug('== begins ==');
logger.info(`process.env.USER = ${process.env.USER}`);
logger.info(`process.env.HOME = ${process.env.HOME}`);
logger.info(`process.env.AAD_TENANT_ID = ${process.env.AAD_TENANT_ID}`);
logger.info(`process.env.AAD_CLIENT_ID = ${process.env.AAD_CLIENT_ID}`);
logger.info(`process.env.AAD_ENDPOINT_HOST = ${process.env.AAD_ENDPOINT_HOST}`);
logger.info(
  `process.env.MSAL_HOME_ACCOUNT_ID = ${process.env.MSAL_HOME_ACCOUNT_ID}`
);
logger.debug('== ends ==');

### Packages
- [fs](https://nodejs.org/docs/latest/api/path.html)
- [path](https://nodejs.org/docs/latest/api/path.html)
- [@azure/msal-node](https://github.com/AzureAD/microsoft-authentication-library-for-js)
- [@microsoft/microsoft-graph-client](https://github.com/microsoftgraph/msgraph-sdk-javascript#readme)

In [ ]:
logger.debug('== begins ==');
import Path from "path";
import FS from "fs";
import * as MSAL from "@azure/msal-node";
import * as fosMSAL from "@iamsupercb/msal";
import { IResultSet, IScenario, ScenarioStatus } from "@iamsupercb/jupyter";
import * as MSGraphClient from '@microsoft/microsoft-graph-client';
logger.debug('== ends ==');

### Globals

In [ ]:
logger.debug("== begins ==");
var resultSet: IResultSet = {
  overwrite: true,
  notebookName: "template",
  description: "template for fos jupyter notebook that uses msal",
  startDatetime: new Date().toISOString(),
  dataPath: Path.join(process.env.HOME, "data", "fos", "typescript", "msgraph"),
  scenarios: [],
};
if (!FS.existsSync(resultSet.dataPath)) {
  FS.mkdirSync(resultSet.dataPath, { recursive: true });
}
logger.debug(`resultSet = ${JSON.stringify(resultSet)}`);
logger.debug("== getToken begins ==");
let pca = await fosMSAL.init(fosMSAL.defaultConfiguration);
if (!pca) {
  throw new Error("pca is null");
}
let tokenCache = await pca.getTokenCache();
let accountInfo = await tokenCache.getAccountByHomeId(
  process.env.MSAL_HOME_ACCOUNT_ID
);
logger.info(`name: ${accountInfo.name}`);
logger.info(`username = ${accountInfo.username}`);
logger.info(`tenantId = ${accountInfo.tenantId}`);
logger.info(`homeAccountId = ${accountInfo.homeAccountId}`);
let token = await pca.acquireTokenSilent({
  scopes: ["https://graph.microsoft.com/User.Read"],
  account: accountInfo,
});
logger.info("resultSet.token acquired...");
logger.debug("== getToken ends ==");
resultSet.msGraphClient = MSGraphClient.Client.init({
  authProvider: async (done) => {
    done(null, token.accessToken);
  },
});
logger.debug("== ends ==");


### Who am I

In [ ]:
logger.debug("== begins ==");
let result = await resultSet.msGraphClient.api("/me").get();
logger.info(`result = ${JSON.stringify(result,null,2)}`);
logger.debug("== ends ==");


## Scenarios

### Sample Scenario

In [ ]:
logger.debug("== begins ==");
let scenario:IScenario = {
  name: "PhotoValue",
  description: "/me/photo/$value",
  startDatetime: new Date().toISOString(),
};
scenario.result = {};
logger.debug(JSON.stringify(scenario.result, null, 2));
scenario.status = ScenarioStatus.Completed;
scenario.endDatetime = new Date().toISOString();
resultSet.scenarios.push(scenario);
logger.debug("== ends ==");


## Finalization

### Summary 

In [ ]:
logger.debug('== begins ==');
resultSet.endDatetime = (new Date()).toISOString();
logger.info(`resultSet.startDatetime = ${resultSet.startDatetime}`);
logger.info(`resultSet.scenarios.length = ${resultSet.scenarios.length}`);
logger.info(`resultSet.endDatetime = ${resultSet.endDatetime}`);
resultSet.endDatetime = (new Date()).toISOString();
logger.debug('== ends ==');

### Cleanup

In [ ]:
logger.debug('== begins ==');
delete resultSet.msGraphClient;
logger.debug('== ends ==');

### Save resultset

In [ ]:
logger.debug('== begins ==');
var resultSetFilename = Path.join(resultSet.dataPath, `resultSet.${resultSet.notebookName}${resultSet.overwrite?'':('.'+resultSet.endDatetime)}.json`);
FS.writeFileSync(resultSetFilename, JSON.stringify(resultSet, null, 2),{encoding:'utf8',flag:'w'});
logger.debug('== ends ==');